In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install -qqq langchain llama-index llama-hub openai accelerate==0.21.0 bitsandbytes==0.40.2 transformers sentence_transformers InstructorEmbedding chromadb pypdf pdf2image

In [ ]:
!pip install transformers

In [ ]:
import torch
import logging
import sys
import os
import pickle


# llama_index
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage, ServiceContext
from langchain.vectorstores import Chroma

from transformers import AutoTokenizer, TextStreamer, pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer


# llama_index
from llama_index.response.notebook_utils import display_response
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor, KeywordNodePostprocessor
from llama_index.response_synthesizers import get_response_synthesizer
from llama_index import SummaryIndex
from llama_index.text_splitter import SentenceSplitter
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.response.pprint_utils import pprint_response
from llama_index.prompts import PromptTemplate
from llama_index.llms import HuggingFaceLLM


# chromadb
import chromadb

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
!pip install sentence-transformers
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from sentence_transformers import SentenceTransformer
import torch

In [ ]:
!pip install PyMuPDF
!pip install docx2txt

In [ ]:
documents = SimpleDirectoryReader('./drive/MyDrive/Chatbot_mining/pdfs').load_data()
len(documents)

7

In [ ]:
from google.colab import userdata

# huggingface api token for downloading llama2
hf_token = userdata.get('hf')

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = HuggingFaceLLM(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    query_wrapper_prompt=PromptTemplate("<s> [INST] {query_str} [/INST] "),
    context_window=3900,
    model_kwargs={"token": hf_token, "quantization_config": quantization_config},
    tokenizer_kwargs={"token": hf_token},
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
!pip uninstall sentence-transformers
!pip install sentence-transformers==2.2.2

Found existing installation: sentence-transformers 2.3.1
Uninstalling sentence-transformers-2.3.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/sentence_transformers-2.3.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/sentence_transformers/*
Proceed (Y/n)? y
  Successfully uninstalled sentence-transformers-2.3.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=435d83b9379877c1e5d293e9a3afd881e53928451806b208f6ddac8e8d007953
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
embed_model = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20) # using the default chunk_# values as they work just fine

# set context window
context_window = 4096
# set number of output tokens
num_output = 256

service_context = ServiceContext.from_defaults(llm=llm,
                                               embed_model=embed_model,
                                               text_splitter=text_splitter,
                                              #  context_window=context_window,
                                              #  num_output=num_output,
                                               )

In [ ]:
vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
summary_index = SummaryIndex.from_documents(documents, service_context=service_context)

In [ ]:
# initialize client, setting path to save data
# db = chromadb.PersistentClient(path="./drive/MyDrive/Mining_chatbot/chroma_db")

# # create collection
# chroma_collection = db.get_or_create_collection("bank_earnings_database")

# # assign chroma as the vector_store to the context
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)

  # create your index
vector_index = VectorStoreIndex.from_documents(
      documents,
      service_context=service_context
  )

In [ ]:

query_engine = vector_index.as_query_engine(response_mode="compact")

response = query_engine.query("What is the significance of the Colliery Control Order, 2000?")

pprint_response(response, show_source=True)
# display_response(response)

Final Response: Based on the context information provided, the
Colliery Control Order, 2000 is a regulatory framework that outlines
guidelines and procedures for coal mining operations in India. The
order aims to ensure safety standards and procedures for coal mines,
covering aspects like ventilation, equipment, and emergency
preparedness. It is a significant document for mining operations in
India as it provides detailed rules and regulations related to coal
mining, ensuring the safety of mine workers and the environment.
______________________________________________________________________
Source Node 1/2
Node ID: dfa553dc-7a2e-4dec-a493-4ae3f990fea3
Similarity: 0.8531368515847979
Text: 1. [Q]: What is the purpose of The Coal Mines Act, 1952?
[A]: The Coal Mines Act, 1952, aims to regulate the working conditions
in coal mines and ensure the safety and welfare of mine workers.    2.
[Q]: Can you provide details on the Indian Explosives Act, 1884?
[A]: The Indian Explosives Act, 1884,

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="compact")
response = query_engine.query("major iron ores in india")
display_response(response)


**`Final Response:`** Based on the context information provided, the answer to the query "major iron ores in India" is:

1. Iron ore mining is a key focus in India due to its crucial role as a raw material for steel production.
2. The major iron ore-producing states in India include Odisha, Chhattisgarh, Karnataka, Jharkhand, and Goa.
3. Prominent iron ore mines in India include Bailadila mines in Chhattisgarh, Donimalai mines in Karnataka, and Dalli-Rajhara mines in Madhya Pradesh.
4. The primary types of mining activities in India include coal mining, iron ore mining, bauxite mining, copper mining, gold mining, limestone mining, gypsum mining, and diamond mining.
5. The Indian government plays a regulatory role in iron ore mining through policies, environmental clearances, and monitoring to ensure sustainable and responsible mining practices.
6. The Mines and Minerals (Development and Regulation) Act, 1957, is the primary

In [ ]:
%%time
query_engine_summarized = vector_index.as_query_engine(response_mode="tree_summarize")

response = query_engine_summarized.query("Tell about the mineral conservation and development rules 2017")

display_response(response)

**`Final Response:`** The Mineral Conservation and Development Rules 2017 are the regulations that govern the mining sector in India. These rules were introduced to ensure the sustainable and responsible extraction, processing, and trading of minerals. Here are some key points related to the rules:

1. Objective: The primary objective of the rules is to balance the conservation of mineral resources with the sustainable development of the mining sector.
2. Applicability: The rules apply to all mining activities, including captive mining, private mining, and public mining.
3. Environmental Clearance: Environmental clearance is a mandatory requirement for all mining projects. The rules outline the criteria for environmental clearance, including the impact assessment of the mining project on the environment.
4. Rehabilitation and Restoration: The rules emphasize the importance of rehabilitation and restoration of mined areas. The mining lessee is required to submit a rehabilitation and restoration plan to ensure the area is restored to its original state after mining activities.
5. Safety Measures: The rules outline safety measures that must be taken to ensure

CPU times: user 21.6 s, sys: 982 ms, total: 22.5 s
Wall time: 22.7 s


In [ ]:
#customizing retireval query
retriever = VectorIndexRetriever(
    index=vector_index,
    similarity_top_k=6,
)

# configure node postprocessors
s_processor = SimilarityPostprocessor(similarity_cutoff=0.79)
k_processor = KeywordNodePostprocessor(
    exclude_keywords=["environmental"]
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(service_context=service_context)

output = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[s_processor, k_processor],
    response_synthesizer=response_synthesizer
)

In [ ]:
response = output.query("Tell about the mineral conservation and development rules 2017?")

display_response(response)

In [ ]:
#chat_engine
chat_engine = vector_index.as_chat_engine(chat_mode="condense_question")

In [ ]:
response = chat_engine.chat("what is dgms circular")
print(response)

In [ ]:
response = chat_engine.chat("largest coal producer in india")
print(response)

In [ ]:
chat_engine = vector_index.as_chat_engine(chat_mode="condense_question")

In [ ]:
response = chat_engine.chat("what are the licensing issues of mining in india")
print(response)

In [ ]:
response = chat_engine.chat("major iron ore-producing states in India")
print(response)

In [ ]:
chat_engine.reset()